In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import copy 
import random
import json

sys.path.insert(1, os.path.realpath(os.path.pardir))

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from node2vec2rank.model import n2v2r
from scipy.sparse import csc_matrix

from sklearn import preprocessing

from scipy import stats

import spectral_embedding as se         

In [2]:
def generate_two_DSBM_graphs_fixed_four_original(num_nodes, multiplicity = 1):
    num_com = 4

    pi = np.repeat(1/num_com, num_com)

    Bs = np.array([[[0.08, 0.02, 0.18, 0.10],
                    [0.02, 0.20, 0.04, 0.10],
                    [0.18, 0.04, 0.02, 0.02],
                    [0.10, 0.10, 0.02, 0.06]],
                [[0.16, 0.16, 0.04, 0.10],
                    [0.16, 0.16, 0.04, 0.10],
                    [0.04, 0.04, 0.09, 0.02],
                    [0.10, 0.10, 0.02, 0.06]]])
    

    list_of_pairs_of_graphs = []
    list_of_assignments = []

    for i in range(multiplicity):

        As, Z = se.generate_SBM_dynamic(num_nodes, Bs, pi)

        colours = np.array(list(mpl.colors.TABLEAU_COLORS.keys())[0:num_com])
        Zcol = colours[Z]
        
        list_of_pairs_of_graphs.append(As)

        le = preprocessing.LabelEncoder()
        assignments = le.fit_transform(Zcol)
        list_of_assignments.append(assignments)
    
    return list_of_pairs_of_graphs, Bs[0], Bs[1], list_of_assignments

def generate_two_DSBM_graphs_random_changes(num_nodes, num_com, max_comm_prob=0.5, multiplicity_model = 1, multiplicity_samples=1):

    list_of_pairs_of_graphs = []
    list_of_assignments = []

    for _ in range(multiplicity_model):

        first_true_memb_matrix = np.random.uniform(0.01, max_comm_prob,(num_com,num_com))
        second_true_memb_matrix = np.random.uniform(0.01, max_comm_prob,(num_com,num_com))

        stacked_true_memb_matrix = np.stack((first_true_memb_matrix, second_true_memb_matrix))

        pi = np.repeat(1/num_com, num_com)

        for _ in range(multiplicity_samples):

            As, Z = se.generate_SBM_dynamic(num_nodes, stacked_true_memb_matrix, pi)

            colours = np.array(list(mpl.colors.TABLEAU_COLORS.keys())[0:num_com])
            Zcol = colours[Z]
            
            list_of_pairs_of_graphs.append(As)

            le = preprocessing.LabelEncoder()
            assignments = le.fit_transform(Zcol)
            list_of_assignments.append(assignments)
    
    return list_of_pairs_of_graphs, stacked_true_memb_matrix[0], stacked_true_memb_matrix[1], list_of_assignments

In [220]:
num_nodes = 2000
num_com = 4
multiplicity_samples = 1

list_of_pairs_of_graphs, first_true_memb_matrix, second_true_memb_matrix, list_of_assignments = generate_two_DSBM_graphs_fixed_four_original(num_nodes=num_nodes, multiplicity=multiplicity_samples)
graph_one = list_of_pairs_of_graphs[0][0]
graph_two = list_of_pairs_of_graphs[0][1]
# np.savetxt("graph_one.csv", graph_one, delimiter=",")
# np.savetxt("graph_two.csv", graph_two, delimiter=",")
# np.savetxt("graph_one_memb.csv", first_true_memb_matrix, delimiter=",")
# np.savetxt("graph_two_memb.csv", second_true_memb_matrix, delimiter=",")
# np.savetxt("node_ass.csv", list_of_assignments, delimiter=",")


In [221]:
from node2vec2rank.visualization_utils import plot_embeddings
_ , Ys=se.UASE(list_of_pairs_of_graphs[0],2)

ys_concat = np.concatenate(Ys)



plot_embeddings(ys_concat)


In [225]:
subsamples=random.sample(list(np.arange(1000)), 60)

ys_concat = np.concatenate(Ys[:,subsamples,:])

plot_embeddings(ys_concat)

In [226]:
np.savetxt("graph_one_UASE_muaha.csv", Ys[0,subsamples,:], delimiter=",")
np.savetxt("graph_two_UASE_muaha.csv", Ys[1,subsamples,:], delimiter=",")
